In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
original_gas_prices_df = pd.read_csv(r'..\\DataSet\\2004-2021.tsv', sep='\t')
original_gas_prices_df['year'] = pd.DatetimeIndex(original_gas_prices_df['DATA INICIAL']).year
df_mask = original_gas_prices_df['year'] < 2020
original_gas_prices_df = original_gas_prices_df[df_mask]

# df_mask = original_gas_prices_df['year'] > 2010
# original_gas_prices_df = original_gas_prices_df[df_mask]
# original_gas_prices_df
# original_gas_prices_df.sample(n=4)

In [ ]:
plt.rcParams['figure.figsize'] = (11,7)
plt.style.use('seaborn')

In [ ]:
original_gas_prices_df['DATA FINAL'] = pd.to_datetime(original_gas_prices_df['DATA FINAL']) # Converte para período, desprezando dia do mês

# Algumas colunas possuem '-' como valor, causando erro ao converter, por isso errors='coerse' seta como NaN
original_gas_prices_df['PREÇO MÉDIO DISTRIBUIÇÃO'] = pd.to_numeric(original_gas_prices_df['PREÇO MÉDIO DISTRIBUIÇÃO'], errors='coerce')
original_gas_prices_df['PREÇO MÉDIO REVENDA'] = pd.to_numeric(original_gas_prices_df['PREÇO MÉDIO REVENDA'], errors='coerce')

# Seta Data final da pesquisa como indíce
original_gas_prices_df.set_index(['DATA FINAL'], inplace=True)
# original_gas_prices_df.head()

In [ ]:
used_columns = [
    'DATA',
    'PRODUTO',
    'ESTADO',
    'REGIÃO',
    'PREÇO MÉDIO DISTRIBUIÇÃO',
    'PREÇO MÉDIO REVENDA',
]

used_products = [
    'ETANOL HIDRATADO',
    'GASOLINA COMUM',
    'ÓLEO DIESEL'
]

monthly_df = original_gas_prices_df.groupby(['PRODUTO', 'ESTADO', 'REGIÃO']).resample('M').mean().reset_index()
monthly_df['DATA'] = monthly_df['DATA FINAL'].dt.to_period('M')

gas_prices_df = monthly_df[used_columns]
gas_prices_df = gas_prices_df[gas_prices_df.PRODUTO.isin(used_products)]
gas_prices_df = gas_prices_df.dropna(how='any')

gas_prices_df['VARIAÇÃO DIST X REV'] = gas_prices_df['PREÇO MÉDIO REVENDA'] - gas_prices_df['PREÇO MÉDIO DISTRIBUIÇÃO']
gas_prices_df['VARIAÇÃO PERCENTUAL DIST X REV'] = gas_prices_df['VARIAÇÃO DIST X REV'] /  gas_prices_df['PREÇO MÉDIO DISTRIBUIÇÃO'] * 100

gas_prices_df.set_index(['DATA', 'PRODUTO', 'ESTADO', 'REGIÃO'], inplace=True)
# gas_prices_df.sample(n=5)

In [ ]:
original_inflation_rate_df = pd.read_csv('..\\DataSet\\inflation-rate-brazil.csv')
# original_inflation_rate_df['date'] = pd.to_datetime(original_inflation_rate_df['date']).dt.to_period('M') 
# inflation_rate_df = original_inflation_rate_df.set_index('date')['2004':'2019']
# inflation_rate_df.to_csv('outputCSV.csv')

# edit_inflation_rate_df = inflation_rate_df.copy()
# edit_inflation_rate_df = edit_inflation_rate_df.reset_index(drop=False)
# edit_inflation_rate_df['date'] = edit_inflation_rate_df['date'].astype('datetime64[ns]')

# df = pd.DataFrame(pd.date_range('2019-07-01', freq='1d', periods=1030))
# df['day'] = pd.DatetimeIndex(df[0]).day
# df_mask = df['day'] == 1
# df = df[df_mask]
# df = df.reset_index(drop=True)

# ipca_df = pd.read_excel(r"C:\Users\chris\OneDrive\Área de Trabalho\ipca_SerieHist\ipca_202204SerieHist.xls")
# ipca_df = ipca_df[378:415]
# ipca_df = ipca_df[ipca_df['Unnamed: 7'].notna()]

# ipca_df = ipca_df.drop(['Unnamed: 0','Unnamed: 1','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'], axis=1)
# # ipca_df['Date'] = df[0]
# #ipca_df = pd.merge(ipca_df, df, how='outer', on=['year', 'semestre'])
# ipca_df = ipca_df.reset_index(drop=True)
# ipca_df.insert(0, "Data", df[0], True)
# ipca_df.insert(2, "2", 0, True)
# ipca_df.insert(3, "3", 0, True)

# for index, row in ipca_df.iterrows():
#     edit_inflation_rate_df=edit_inflation_rate_df.append({'date' : row['Data'] , 'absolute_index' : row['Unnamed: 2'], 'monthly' : 0, 'semiannual_accumulation' : 0, 'annual_accumulation' : row['Unnamed: 7']} , ignore_index=True)
# edit_inflation_rate_df = edit_inflation_rate_df.drop_duplicates(subset='date', keep='first')
# inflation_rate_df = edit_inflation_rate_df.set_index('date')['2004':'2021-04']

original_inflation_rate_df['date'] = pd.to_datetime(original_inflation_rate_df['date']).dt.to_period('M') 
inflation_rate_df = original_inflation_rate_df.set_index('date')['2004':'2019']
# inflation_rate_df.sample(n=5)



In [ ]:
# Mesclando inflação + preços
main_df = gas_prices_df.copy()
main_df.reset_index(inplace=True)
main_df.set_index('DATA', inplace=True)
main_df['INFLAÇÃO ANUAL'] = inflation_rate_df['annual_accumulation']  # INFLAÇÃO ANUAL ACUMULADA NOS ÚLTIMOS 12 MESES
main_df['INFLAÇÃO ABSOLUTA'] = inflation_rate_df['absolute_index']  # INFLAÇÃO ANUAL ACUMULADA NOS ÚLTIMOS 12 MESES

main_df = main_df.to_timestamp()
main_df = main_df['2006':]

# main_df.sample(5)

In [ ]:
def plot_prices_correlation(produto):
    grp = main_df[main_df.PRODUTO == produto].groupby(['DATA']).mean()
    fig, ax = plt.subplots()

    grp.plot(y='PREÇO MÉDIO REVENDA', label='Revenda', ax=ax)
    grp.plot(y='PREÇO MÉDIO DISTRIBUIÇÃO', label='Distribuição', ax=ax)
    fig.suptitle('Preços de Distribuição e Revenda - ' + produto)
    ax.set_xlabel('Data')
    ax.set_ylabel('Preço - R$/l')
    ax.grid(True)
#     ax.annotate('Teste', xy= ('2018-05-21'), xycoords='data',
#              xytext=(-30, 0))
    
    # plot variação entre distribuição e revenda
    ax_var = ax.twinx()

    grp.plot(y='VARIAÇÃO PERCENTUAL DIST X REV', label='Variação - Distribuição', ax=ax_var, c='#c44e52')
    ax_var.set_ylabel('% - Variação entre Distribuição e Revenda', color='#c44e52')
    ax_var.get_legend().remove()
    ax_var.grid(False)
    plt.show()

# plot_prices_correlation('GASOLINA COMUM')

In [ ]:


def plot_inflation_correlation(produto):
    annual_price_change_df = main_df[main_df.PRODUTO == produto].groupby(['DATA']).mean()
    annual_price_change_df['VARIAÇÃO'] = annual_price_change_df['PREÇO MÉDIO REVENDA'].pct_change()
    annual_price_change_df['VARIAÇÃO 12 MESES'] = annual_price_change_df['VARIAÇÃO'].rolling(min_periods=12, window=12).sum() * 100
    annual_price_change_df.tail()
    
    # ploting
    fig, ax_gas = plt.subplots()
    annual_price_change_df.plot(y='VARIAÇÃO 12 MESES', c='#4c72b0', ax=ax_gas)
    fig.suptitle('Variação da Inflação e Preço Acumulado - ' + produto)
    ax_gas.set_xlabel('Data de pesquisa')
    ax_gas.set_ylabel('% - Variação Acumulativa - 12 meses', color='#4c72b0')
    ax_gas.get_legend().remove()
    ax_gas.grid(True)
    
    # plot_inflation
    ax_inflation = ax_gas.twinx()
    
    annual_price_change_df.plot(y='INFLAÇÃO ANUAL', ax=ax_inflation, c='#55a868')
    ax_inflation.set_ylabel('% - Inflação Acumulada - 12 meses', color='#55a868')
    ax_inflation.get_legend().remove()
    ax_inflation.grid(False)
    plt.show()

# plot_inflation_correlation('GASOLINA COMUM')



In [ ]:
def plot_avg_region(produto):
    avg_per_region_df = main_df['2009':].groupby(['PRODUTO', 'REGIÃO', 'DATA']).mean()
    avg_per_region_for_product_df = avg_per_region_df.iloc[avg_per_region_df.index.get_level_values('PRODUTO') == produto]
    
    fig, ax = plt.subplots()
    fig.suptitle('Preço por Região - ' + produto)
    
    for key, grp in avg_per_region_for_product_df.groupby('REGIÃO'):
        grp_as_timeseries = grp.reset_index().set_index('DATA')
        grp_as_timeseries.plot(y='PREÇO MÉDIO REVENDA', label=key, ax=ax)

        ax.set_xlabel('Data')
        ax.set_ylabel('Preço - R$/l')

    plt.grid(True)
    plt.show()

# plot_avg_region('GASOLINA COMUM')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression

In [ ]:
models = {}
def regression_for_product(produto):
    # Cria Daframe necessário para regressão
    reg_df = main_df['2009':].groupby(['PRODUTO', 'DATA']).mean()
    reg_df = reg_df.iloc[reg_df.index.get_level_values('PRODUTO') == produto].groupby('DATA').mean()[['PREÇO MÉDIO REVENDA', 'INFLAÇÃO ABSOLUTA']]

    # Separa eixos da regressão
    
    reg_df = reg_df.dropna()
    #print(reg_df)
    X = X = reg_df[['INFLAÇÃO ABSOLUTA']]
    Y = reg_df['PREÇO MÉDIO REVENDA'].values

    # Divide dados entre treinamento e teste
    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.20)

    # Cria model de regressão e faz o treinamento
    lr=LinearRegression()
    lr.fit(x_train,y_train)
    models[produto] = lr
    predicted_values = []
    for i in range(0, len(y_test)):
        predicted_values.append(lr.predict(x_test.iloc[[i],:])[0])

    predicted_df = pd.DataFrame({'Inflação Absoluta':x_test['INFLAÇÃO ABSOLUTA'].values , 'Valor Real':y_test, 'Valor Predito':predicted_values})
    fig, ax = plt.subplots()
    fig.suptitle('Regressão - ' + produto)
    ax.set_ylabel('Preço Revenda - R$/l')
    predicted_df.sort_values(by=['Inflação Absoluta']).set_index('Inflação Absoluta').plot(ax=ax)
    #plt.show()

In [ ]:
regression_for_product('GASOLINA COMUM')

In [ ]:
def predict_price(produto):
    rates = {}
    rates['2019-08-01'] = inflation_rate_df['absolute_index'].iloc[-1]
    for i in range(2020, 2029):
        last_year = str(i - 1)+'-08-01'
        curr_year = str(i)+'-08-01'
        rates[curr_year]  = (rates[last_year] * 0.04) + rates[last_year]

    rates_df = pd.DataFrame(list(rates.items()), columns = ['DATA', 'INFLAÇÂO ABSOLUTA'])

    rates_df['DATA'] = pd.to_datetime(rates_df['DATA'])
    rates_df.set_index('DATA', inplace=True)
    predictions = {}

    rates_df['PREVISÃO'] = models[produto].predict(rates_df)

    return rates_df

In [ ]:
predict_price('GASOLINA COMUM')